In [1]:
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd


# 生成初始拉丁超立方设计
def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init


# 退火参数初始化
def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax


# 成本函数，最大化样本间最小距离
def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist


# 生成邻域解（改进版）
def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':
        # 仅扰动Q列（第0列）
        swap_indices = random.sample(range(n), max(1, n // 10))  # 扰动10%的样本
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'R_only':
        # 仅扰动R列（第1列）
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'both':
        # 同时扰动两个维度
        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])

    return X_candidate


# Metropolis接受准则
def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)


# 终止条件判定
def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False


# 模拟退火算法（增加模式参数）
def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0

    return X_current


# 确保目标目录存在
output_dir = "sample"
os.makedirs(output_dir, exist_ok=True)  # 自动创建目录（若不存在）

# Q不变情况下的R
# 1. 定义参数
sim_name = "mf6"
length_units = "meters"
time_units = "days"

# 文档参数
nper = 20  # 总模拟时间 10 年，分 20 个应力周期
nlay = 1
nrow = 75  # 假设值，需根据实际调整
ncol = 125  # 根据代码中最大列数调整
delr = 20.0
delc = 20.0
top = 100.0  # 假设值，需根据实际调整
botm = np.full((nlay, nrow, ncol), 60.0)  # 所有单元格层底高程为70m
icelltype = 1
strt = np.full((nlay, nrow, ncol), 70.0)  # 所有单元格初始水头40m
laytyp = [1, 0]  # 潜水层+承压层
# 定水头边界条件
chd_spd = {
    0: [
        [0, row, 15, 100] for row in range(0, 5)
    ] + [
        [0, 5, col, 100] for col in range(10, 16)
    ] + [
        [0, row, 10, 100] for row in range(5, 11)
    ] + [
        [0, 15, col, 100] for col in range(5, 10)
    ] + [
        [0, row, 10, 100] for row in range(10, 16)
    ] + [
        [0, 20, col, 100] for col in range(0, 5)
    ] + [
        [0, row, 5, 100] for row in range(15, 21)
    ] + [
        [0, row, 104, 80] for row in range(69, 75)
    ] + [
        [0, 69, col, 80] for col in range(105, 110)
    ] + [
        [0, row, 109, 80] for row in range(64, 69)
    ] + [
        [0, 64, col, 80] for col in range(110, 115)
    ] + [
        [0, row, 114, 80] for row in range(59, 65)
    ] + [
        [0, 59, col, 80] for col in range(115, 120)
    ] + [
        [0, row, 119, 80] for row in range(54, 60)
    ] + [
        [0, 54, col, 80] for col in range(120, 125)
    ] 
}

unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")

chd_spd = unique_chd_spd

# 水头边界和渗透系数分布相关代码
# 初始化 idomain 数组，所有单元格默认为活动状态
idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)

# 原离散区域，修正了缺少逗号的问题
original_regions = [
    (0, 0, *range(0, 15)),
    (0, 5, *range(0, 10)),
    (0, 10, *range(0, 10)),
    (0, 15, *range(0, 5)),
    (0, 20, 0),
    (0, 30, *range(0, 5)),
    (0, 35, *range(0, 5)),
    (0, 40, *range(0, 10)),
    (0, 45, *range(0, 15)),
    (0, 50, *range(0, 20)),
    (0, 55, *range(0, 60)),
    (0, 60, *range(0, 65)),
    (0, 65, *range(0, 80)),
    (0, 70, *range(0, 85)),
    (0, 74, *range(0, 95))
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])

# 标记非活动单元格
for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1

# 让 20 到 25 行保持活动状态
for row in range(20, 25):
    idomain[0, row, :5] = 1

original_regions = [
    (0, 0, *range(40, 125)),
    (0, 5, *range(50, 125)),
    (0, 10, *range(75, 125)),
    (0, 15, *range(80, 125)),
    (0, 20, *range(105, 125)),
    (0, 25, *range(115, 125)),
    (0, 30, *range(120, 125)),
    (0, 60, *range(120, 125)),
    (0, 65, *range(115, 125)),
    (0, 70, *range(110, 125)),
    (0, 74, *range(105, 125)),
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])

# 标记非活动单元格
for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1

# 让 35 到 55 行保持活动状态
for row in range(35, 55):
    idomain[0, row, 120:] = 1


# 初始化 hk 数组（原尺寸）
hk_original = np.ones((nlay, nrow, ncol), dtype=np.float32)

# 原位置信息赋值
hk_original[:, 0, 3:8] = 0.0004 * 86400
hk_original[:, 1, 2:10] = 0.0004 * 86400
hk_original[:, 2, 2:9] = 0.0004 * 86400
hk_original[:, 3, 1:8] = 0.0004 * 86400
hk_original[:, 4, 0:7] = 0.0004 * 86400
hk_original[:, 5, 1:6] = 0.0004 * 86400
hk_original[:, 6, 1:5] = 0.0004 * 86400
hk_original[:, 7, 2:4] = 0.0004 * 86400

hk_original[:, 2, 9:15] = 0.0002 * 86400
hk_original[:, 3, 8:16] = 0.0002 * 86400
hk_original[:, 4, 7:21] = 0.0002 * 86400
hk_original[:, 5, 7:23] = 0.0002 * 86400
hk_original[:, 6, 10:23] = 0.0002 * 86400
hk_original[:, 7, 14:21] = 0.0002 * 86400
hk_original[:, 8, 16:19] = 0.0002 * 86400

hk_original[:, 5, 6] = 0.0001 * 86400
hk_original[:, 6, 5:10] = 0.0001 * 86400
hk_original[:, 7, 4:12] = 0.0001 * 86400
hk_original[:, 8, 3:11] = 0.0001 * 86400
hk_original[:, 9, 4:10] = 0.0001 * 86400

hk_original[:, 7, 12:14] = 0.0003 * 86400
hk_original[:, 8, 11:16] = 0.0003 * 86400
hk_original[:, 9, 10:17] = 0.0003 * 86400
hk_original[:, 10, 12:18] = 0.0003 * 86400
hk_original[:, 11, 13:19] = 0.0003 * 86400
hk_original[:, 12, 16:19] = 0.0003 * 86400
hk_original[:, 13, 17:20] = 0.0003 * 86400
hk_original[:, 14, 19] = 0.0003 * 86400

hk_original[:, 6, 23] = 0.0007 * 86400
hk_original[:, 7, 21:25] = 0.0007 * 86400
hk_original[:, 8, 19:25] = 0.0007 * 86400
hk_original[:, 9, 16:25] = 0.0007 * 86400
hk_original[:, 10, 17:25] = 0.0007 * 86400
hk_original[:, 11, 18:24] = 0.0007 * 86400
hk_original[:, 12, 18:23] = 0.0007 * 86400
hk_original[:, 13, 19:22] = 0.0007 * 86400
hk_original[:, 14, 20] = 0.0007 * 86400

# 初始化新的 hk 数组（新尺寸）
hk = np.ones((nlay, nrow, ncol), dtype=np.float32)

# 计算行列缩放比例
row_scale = 5
col_scale = 5

# 遍历原 hk 数组，按比例复制到新数组
for lay in range(nlay):
    for row in range(15):
        for col in range(25):
            new_row = row * row_scale
            new_col = col * col_scale
            hk[lay, new_row:new_row + row_scale, new_col:new_col + col_scale] = hk_original[lay, row, col]

num_stress_periods = 20  # 应力期数量
# 河流边界（关键修改部分）
# R变化的情况
rbot = np.linspace(90., 80.25, num=nrow)
# rstage = np.linspace(90.1, 81.25, num=nrow)
riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [35]:
        for row in range(10, 55):
            if idomain[0, row, col] == 1:
                s = rstage[row - 10]
                b = rbot[row - 10]  # 河底高程根据行号取值
                riv_spd[sp].append([0, row, col, s, 50, b])



# 应力周期长度 6 个月
perlen = 6 * 30  # 定义单个应力期（Stress Period）的长度
nstp = 1  # 表示每个应力期仅使用一个时间步，即全应力期长度（180天）作为一个时间步
tsmult = 1  # =1：时间步长恒定（每步长度相同）。>1：时间步长逐步增大（如指数增长）。<1：时间步长逐步减小（较少使用）。

# 定义观测井位置
obs_wells = [
    [0, 15, 20],  # 层，行，列
    [0, 10, 40],
    [0, 10, 70],
    [0, 35, 75],
    [0, 30, 45],  # 层，行，列
    [0, 45, 30],
    [0, 35, 60],
    [0, 65, 95],
    [0, 45, 95],
    [0, 50, 115]
]

# 渗透系数列表
permeability_values = [0.0004 * 86400, 0.0002 * 86400, 0.0001 * 86400, 0.0003 * 86400, 0.0007 * 86400]

# 初始化垂向补给率和蒸发率数组（仅作为基础框架，值将在应力期中动态生成）
recharge = np.zeros((nlay, nrow, ncol))
evaporation = np.zeros((nlay, nrow, ncol))

# 创建 rch 和 evt 模块的应力期数据
rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  # 丰水期
# 生成丰水期垂向补给率随机变量
        recharge_factors = {
            0.0004 * 86400: np.random.uniform(0.016 * 1.1, 0.016 * 1.3),
            0.0002 * 86400: np.random.uniform(0.012 * 1.1, 0.012 * 1.3),
            0.0001 * 86400: np.random.uniform(0.008 * 1.1, 0.008 * 1.3),
            0.0003 * 86400: np.random.uniform(0.014 * 1.1, 0.014 * 1.3),
            0.0007 * 86400: np.random.uniform(0.018 * 1.1, 0.018 * 1.3)
            }
# 生成丰水期蒸发率随机变量
        evaporation_factors = {
            0.0004 * 86400: np.random.uniform(0.002 * 0.7, 0.002 * 0.9),
            0.0002 * 86400: np.random.uniform(0.003 * 0.7, 0.003 * 0.9),
            0.0001 * 86400: np.random.uniform(0.001 * 0.7, 0.001 * 0.9),
            0.0003 * 86400: np.random.uniform(0.004 * 0.7, 0.004 * 0.9),
            0.0007 * 86400: np.random.uniform(0.005 * 0.7, 0.005 * 0.9)
            }
    else:  # 枯水期
# 生成枯水期垂向补给率随机变量
        recharge_factors = {
            0.0004 * 86400: np.random.uniform(0.016 * 0.7, 0.016 * 0.9),
            0.0002 * 86400: np.random.uniform(0.012 * 0.7, 0.012 * 0.9),
            0.0001 * 86400: np.random.uniform(0.008 * 0.7, 0.008 * 0.9),
            0.0003 * 86400: np.random.uniform(0.014 * 0.7, 0.014 * 0.9),
            0.0007 * 86400: np.random.uniform(0.018 * 0.7, 0.018 * 0.9)
            }
# 生成枯水期蒸发率随机变量
        evaporation_factors = {
            0.0004 * 86400: np.random.uniform(0.002 * 1.1, 0.002 * 1.3),
            0.0002 * 86400: np.random.uniform(0.003 * 1.1, 0.003 * 1.3),
            0.0001 * 86400: np.random.uniform(0.001 * 1.1, 0.001 * 1.3),
            0.0003 * 86400: np.random.uniform(0.004 * 1.1, 0.004 * 1.3),
            0.0007 * 86400: np.random.uniform(0.005 * 1.1, 0.005 * 1.3)
            }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()

    for k in permeability_values:
        rch_mask = np.isclose(hk, k)
        current_rch[rch_mask] = recharge_factors[k]

        evt_mask = np.isclose(hk, k)
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch
    evt_spd[sp] = []
    for lay in range(nlay):
        for row in range(nrow):
             for col in range(ncol):
                 if current_evt[lay, row, col] > 0:
                    lay = int(lay)
                    row = int(row)
                    col = int(col)
                    rate = float(current_evt[lay, row, col])
                    # 假设 surface 为 100.0（可根据实际情况修改）
                    surface = 100.0
                    # 假设 depth 为 0.0（可根据实际情况修改）
                    depth = 5.0
                    evt_spd[sp].append([lay, row, col, surface, rate, depth])
                     

# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

if __name__ == "__main__":
    n_Q = 4  # Q 的样本数量
    d_Q = 10  # Q 的维度
    bounds_Q = [(50, 100)] * d_Q  # Q 的边界

    n_R_initial = 1  # 初始水位采样数量
    d_Ri = 20  # R 的维度
    bounds_Ri = [(90.10, 92.50)] * d_Ri  # R 的边界

    n_R_final = 1  # 末尾水位采样数量
    d_Rf = 20  # R 的维度
    bounds_Rf = [(81.25, 82.50)] * d_Rf  # R 的边界

    # 重复运行200次
    for run in range(200):
        # 生成初始拉丁超立方设计
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)

        # 初始化退火参数
        T0, alpha, Nmax = initialize_annealing_parameters()

        # 单独对 Q 进行采样优化
        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized

        # 单独对初始水位 R 进行采样优化
        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized

        # 单独对末尾水位 R 进行采样优化
        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized

        # 确保初始水位数组中每个元素都大于末尾水位数组对应位置的元素
        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)

        # 井采样部分
        n_wells = 10
        # 井采样的维度为 3（层、行、列）
        d_well = 3
        # 这里假设 nlay, nrow, ncol 是你的模型网格的层数、行数、列数
        nlay = 1
        nrow = 75
        ncol = 125
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
        # 生成初始拉丁超立方设计
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)
        # 对井的位置进行取整
        X_init_well = np.round(X_init_well).astype(int)
        # 过滤掉不在活动单元格内的位置
        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)
        # 将有效位置转换为 numpy 数组
        X_init_well = np.array(valid_well_locations)

        # 初始化退火参数
        T0, alpha, Nmax = initialize_annealing_parameters()

        # 对井的位置进行退火拉丁超立方采样优化
        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')
        # 再次对优化后的位置进行取整
        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 
                0.6388 , 0.5806, 0.6135, 0.6934, 0.9771, 0.8470 ])

        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }

        # 将Q采样数据代入前四个应力周期的wel_spd
        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)
        
        # 2. 创建模拟
        sim_ws = os.path.join('conc_models', f'{sim_name}_{run}')  # 定义模型的工作目录（工作空间）
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='mf6.6.1_linux/bin/mf6')  # 创建MODFLOW 6模拟的主对象

        # 应力周期长度 6 个月
        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)]  # 生成时间离散化数据，包含nper=20个元组的列表。
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)  # （对象，总应力期数量，列表每个元素为(perlen, nstp, tsmult)元组，时间单位）

        # 3. 建立模型
        gwfname =  f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", 
                                   save_flows=True)  # newtonoptions="NEWTON UNDER_RELAXATION"​​牛顿-拉夫森法的松弛选项，用于改善非线性问题的收敛性
        # 渗流模型求解参数
        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
          )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)  # 配置模型的网格离散化参数

        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk)  # 定义含水层的水力传导系数（渗透系数）和单元类型

        flopy.mf6.ModflowGwfic(gwf, strt=strt)  # 设置模型的初始水头分布。

        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)  # 定义河流边界条件。

        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], 
                                pname="WEL-1", save_flows=True)  # 定义抽水/注水井

        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)  # 定义垂向补给（如降水入渗）
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)  # 定义定水头边界（如河流或湖泊）
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)
       # 输出数据
       # headfile = os.path.join(run_output_dir, f"gwf_{sim_name}_{run+1}.hds")
        #budgetfile = os.path.join(run_output_dir, f"gwf_{sim_name}_{run+1}.cbb")
        
       # concentrationfile = os.path.join(run_output_dir, f"gwt_{sim_name}_{run+1}.ucn")
        #gwt_budgetfile = os.path.join(run_output_dir, f"gwt_{sim_name}_{run+1}.cbc")
        
        headfile = f"{sim_name}_{run}.hds"
        head_filerecord = [headfile]
        budgetfile = f"{sim_name}_{run}.bud"
        budget_filerecord = [budgetfile]
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=head_filerecord,
                               budget_filerecord=budget_filerecord, printrecord=saverecord)

    # 创建观测井文件
        obs_data = {
            f'obs_well_{i+1}.csv': [
                (f'well_{i+1}', 'HEAD', (lay, row, col))
            ]
            for i, (lay, row, col) in enumerate(obs_wells)
        }

        flopy.mf6.ModflowUtlobs(
              gwf,
              digits=10,
              print_input=True,
              continuous=obs_data,          
        )

    # 创建运移模型并与流动模型耦合
        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")

        # 溶质运移模型求解参数
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])

    # 运移模型的时间离散化（需要与GWF模型同步）
        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)

    # 运移模型的初始条件
        initial_concentration = 0.0  # 初始浓度设为0
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)

    # 弥散系数设置
        dsp = flopy.mf6.ModflowGwtdsp(gwt,
                            alh=40.0,  # 纵向弥散度
                            ath1=4.0,  # 横向弥散度
                            atv=0.1,   # 垂向弥散度
                            diffc=0.0) # 分子扩散系数
    # 运移过程设置
        mst = flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)  # 孔隙度设为0.3
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        
        flopy.mf6.ModflowGwtssm(
              gwt,
              sources = sourcerecarray,
              print_flows=True,
              filename=f"{gwtname}.ssm"
         )


    # 观测井配置（在原有观测井之后添加）
        gwt_obs_package_name = f"gwt_{sim_name}_{run+1}_obs"
        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]

        flopy.mf6.ModflowUtlobs(
              gwt,
              pname=gwt_obs_package_name,
              digits=10,
              print_input=True,
              continuous=gwt_obs,

          )


   # 创建每次运行对应的独立输出文件夹
        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

    # 输出控制
        concentrationfile = f"{sim_name}_{run}.ucn"  # 绝对路径
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt,
                               concentration_filerecord = [concentrationfile],
                               budget_filerecord = [gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), 
                                           ("BUDGET", "ALL")])
    # 耦合模型
        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname,filename="{}.gwfgwt".format(sim_name))
        # 运行模型
        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")

    #     # 结果可视化
        
    #     all_heads = gwf.oc.output.head().get_alldata()
    #     np.save(os.path.join(run_output_dir, f'head_data_{run}.npy'), all_heads)

    #     all_conc = gwt.oc.output.concentration().get_alldata()
    #     np.save(os.path.join(run_output_dir, f'conc_data_{run}.npy'), all_conc)
    
    #     INVALID_VALUE = 1e30
    #     valid_values = []
    #     for kper in range(nper):
    #         data = all_heads[kper][0].copy()
    #         valid_mask = (data < INVALID_VALUE)
    #         valid_values.extend(data[valid_mask].flatten())

    #     global_vmin = np.min(valid_values)
    #     global_vmax = np.max(valid_values)

    #     cmap = plt.cm.viridis.copy()
    #     cmap.set_bad('white', 1.0)

    #     valid_conc_values = []
    #     for kper in range(nper):
    #         data = all_conc[kper][0].copy()
    #         valid_mask = (data < INVALID_VALUE)
    #         valid_conc_values.extend(data[valid_mask].flatten())
            
    #     global_cmin = np.min(valid_conc_values)
    #     global_cmax = np.max(valid_conc_values)


    # # 水头可视化
    #     plt.figure(figsize=(15, 10), facecolor='none')
    #     for kper in range(nper):
    #         plt.subplot(4, 5, kper + 1)
    #         data = all_heads[kper][0].copy()
    #         data[data >= INVALID_VALUE] = np.nan
    #         im = plt.imshow(
    #               data,
    #               cmap=cmap,
    #             vmin=global_vmin,
    #             vmax=global_vmax
    #         )
    #         # **添加等值线**
    #         levels = np.linspace(global_vmin, global_vmax, 20)  # 定义等值线层级（10层）
    #         contour = plt.contour(
    #                 data,
    #                 levels=levels,
    #                 colors='white',  # 等值线颜色
    #                 linewidths=0.7,   # 线宽
    #                 linestyles='solid'  # 线型
    #                 )
    #         plt.clabel(contour, fontsize=6, inline=True)  # 添加等值线标签
    #         plt.title(f"Stress Period {kper + 1}")
    #         plt.colorbar()
    #     plt.tight_layout()
    #     plt.savefig(os.path.join(run_output_dir, f'head_visualization_{run}.png'))
    #     plt.close()

    #     # 浓度可视化
    #     plt.figure(figsize=(15, 10), facecolor='none')
    #     for kper in range(nper):
    #         plt.subplot(4, 5, kper + 1)
    #         data = all_conc[kper][0].copy()
    #         data[data >= INVALID_VALUE] = np.nan
    #         im = plt.imshow(data,
    #                      cmap='jet',
    #                      vmin=global_cmin,
    #                      vmax=global_cmax) 
    #         # **指定等值线数值和颜色**
    #         specified_levels = [0.5, 1.5, 2.0]  # 固定等值线数值
    #         colors = ['red', 'white', 'yellow']     # 对应颜色（与levels顺序一致）
    
    # # 绘制等值线
    #         contour = plt.contour(
    #             data,
    #             levels=specified_levels,
    #             colors=colors,        # 按顺序分配颜色
    #             linewidths=1.0,       # 线宽
    #             linestyles='solid'    # 线型
    #             )
    
    # # 添加等值线标签（仅显示数值，不显示颜色条）
    #         plt.clabel(contour, fmt='%1.1f', fontsize=6, inline=True)  # fmt控制标签格式
    #         plt.title(f"应力期 {kper + 1}")
    #         plt.colorbar(label='浓度 (mg/L)')
    #         plt.title(f"SP {kper + 1}")
    #         plt.colorbar(label='Concentration (mg/L)')
    #     plt.tight_layout()
    #     plt.savefig(os.path.join(run_output_dir, f'conc_visualization_{run}.png'))
    #     plt.close()
       #将本次运行生成的文件移动到对应的独立输出文件夹
        files_to_move = [
            f"optimized_Q_values_{run}.txt",
            f"optimized_R_initial_values_{run}.txt",
            f"optimized_R_final_values_{run}.txt",
            f"well_locations_{run}.txt",
        ]
        for file in files_to_move:
            src_path = os.path.join(output_dir, file)
            dst_path = os.path.join(run_output_dir, file)
            if os.path.exists(src_path):
                os.rename(src_path, dst_path)

已跳过重复的定水头单元格: (0, 5, 10)
已跳过重复的定水头单元格: (0, 10, 10)
已跳过重复的定水头单元格: (0, 15, 5)
已跳过重复的定水头单元格: (0, 64, 114)
已跳过重复的定水头单元格: (0, 59, 119)
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing solution package ims_0...
  writing package mf6.gwfgwt...
  writing model gwf_mf6_0...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package riv-1...
INFORMATION: maxbound in ('gwf6', 'riv', 'dimensions') changed to 41 based on size of stress_period_data
    writing package wel-1...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 10 based on size of stress_period_data
    writing package rcha_0...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 77 based on size of stress_period_data
    writing package evt-1...
INFORMATION: maxbound in ('gwf6', 'evt', 'dimensions') changed to 5275 

FileNotFoundError: The program mf6.6.1_linux/bin/mf6 does not exist or is not executable.